In [18]:
import os
from numpy import load
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.neighbors import KDTree
import time

In [19]:
feature_list = load(r'npy\feature_list.npy')
feature_id = load(r'npy\feature_id.npy')
feature_id = feature_id.tolist()

model = EfficientNetB7(include_top=False)
kd = KDTree(feature_list)

In [20]:
def feature_extract(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    flattened_features = features.flatten()
    normalized_features = flattened_features / np.linalg.norm(flattened_features)
    return normalized_features.reshape(-1,1).T[0]

In [21]:
pathX = 'media\images\dataX\oxbuild_images-v1'
pathY = 'media\images\dataY\gt_files_170407'
query_files = os.listdir(pathY)

In [29]:
all_times = 0
for i in range(3, len(query_files), 4):
    query = open(pathY + '\\' + query_files[i]).read()[:-1].split(' ')
    x = int(float(query[1]))
    y = int(float(query[2]))
    w = int(float(query[3]))
    h = int(float(query[4]))
    good_results = open(pathY + '\\' + query_files[i-3]).readlines()
    ok_results = open(pathY + '\\' + query_files[i-1]).readlines()
    #junk_results = open(pathY + '\\' + query_files[i-2]).readlines()

    accept_results = good_results + ok_results #+ junk_results
    accept_results = [i[:-1] for i in accept_results]

    print('Query:', query[0])

    img = cv2.imread(pathX + '\\' + query[0][5:] + '.jpg')
    cv2.imwrite('cut\\' + query[0][5:] + '.jpg', img[y:y+h, x:x+w])

    start = time.time()
    dist, indx = kd.query(feature_extract('cut\\' + query[0][5:] + '.jpg', model).reshape(-1,1).T, len(feature_list))
    end = time.time()
    all_times = all_times + (end-start)

    index = []
    distance = []
    label = []
    for j in range(len(feature_list)):
        index.append(j+1)
        distance.append(dist[0][j])
        label.append(1) if feature_id[indx[0][j]].split('/')[-1][:-4] in accept_results else label.append(0)

    df = pd.DataFrame(data={"index": index, "distance": distance, "label": label})
    df.to_csv('retrieval_query_results/query_' + query[0] + '.csv', index=False)

Query: oxc1_all_souls_000013
1/1 [==============================] - 0s 297ms/step
Query: oxc1_all_souls_000026
1/1 [==============================] - 0s 270ms/step
Query: oxc1_oxford_002985
1/1 [==============================] - 0s 269ms/step
Query: oxc1_all_souls_000051
1/1 [==============================] - 0s 274ms/step
Query: oxc1_oxford_003410
1/1 [==============================] - 0s 274ms/step
Query: oxc1_ashmolean_000058
1/1 [==============================] - 0s 260ms/step
Query: oxc1_ashmolean_000000
1/1 [==============================] - 0s 265ms/step
Query: oxc1_ashmolean_000269
1/1 [==============================] - 0s 295ms/step
Query: oxc1_ashmolean_000007
1/1 [==============================] - 0s 271ms/step
Query: oxc1_ashmolean_000305
1/1 [==============================] - 0s 273ms/step
Query: oxc1_balliol_000051
1/1 [==============================] - 0s 267ms/step
Query: oxc1_balliol_000187
1/1 [==============================] - 0s 249ms/step
Query: oxc1_balliol_000167

In [30]:
all_times

64.76725721359253